<a href="https://colab.research.google.com/github/AdyAhmadi24/Fraud/blob/main/Tugas_Akhir_Asli_fix_Ady_Ahmadi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTING PACKAGES
from google.colab import drive
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn import preprocessing
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier # Decision tree algorithm
from sklearn.model_selection import train_test_split # Data Splitting
from sklearn.preprocessing import StandardScaler # Data Standadization
from sklearn.preprocessing import MinMaxScaler #Min-Max Data Normalization
from sklearn.ensemble import RandomForestClassifier # Random forest tree algorithm
from sklearn.linear_model import LogisticRegression # Logistic regression algorithm
from imblearn.over_sampling import ADASYN # Oversampling Data with ADASYN

In [ ]:
#IMPORTING DATA
url = 'https://raw.githubusercontent.com/AdyAhmadi24/Fraud/main/Flagging_Witel_202012.csv'
df = pd.read_csv(url)

#Check the columns name
df.columns.values.tolist()

['NO',
 'IS_FRAUD',
 'DATA CUTOFF',
 'NCLI',
 'ND_INET',
 'REG',
 'WITEL',
 'DATEL',
 'STO',
 'CITEM_SPEED',
 'SPEED_INET',
 'STATUS_INET',
 'USAGE',
 'BILLING',
 'UMUR_LANGGANAN',
 'PRA_PASCA',
 'FLAGGING']

In [ ]:
#Removing unnecessary column
uc = ['IS_FRAUD', 'NO', 'DATA CUTOFF', 'NCLI', 'ND_INET', 'REG', 'WITEL', 'DATEL', 'CITEM_SPEED','STATUS_INET']
df.drop(uc, inplace=True, axis=1)
df

,STO,SPEED_INET,USAGE,BILLING,UMUR_LANGGANAN,PRA_PASCA,FLAGGING
0,MAT,20M,689.73,483300,15,PASCABAYAR,NaN
1,PNK,10M,404.09,217100,36,PASCABAYAR,NaN
2,SIN,10M,373.29,268700,4,PASCABAYAR,NaN
3,SUG,20M,255.92,391000,1,PASCABAYAR,NaN
4,TMA,50M,851.28,771800,51,PASCABAYAR,NaN
...,...,...,...,...,...,...,...
136238,BAL,200M,2406.40,1949600,15,PASCABAYAR,NaN
136239,KIM,100M,2535.69,944100,19,PASCABAYAR,NaN
136240,PNK,10M,410.95,279400,38,PASCABAYAR,NaN
136241,PNK,40M,697.03,552400,60,PASCABAYAR,NaN


In [ ]:
#Rename classes column
df.rename(columns={'FLAGGING': 'Class'}, inplace=True)

#Filling classes column with int value
df['Class'] = df['Class'].fillna(0)
df['Class'] = df['Class'].replace(['fraud'], 1)

#Summarizing fradulent cases
cases = len(df)
nonfraud_count = len(df[df.Class == 0])
fraud_count = len(df[df.Class == 1])
fraud_percentage = round(fraud_count/nonfraud_count*100, 2)

print('CASE COUNT')
print('--------------------------------------------')
print('Total number of cases are {}'.format(cases))
print('Number of Non-fraud cases are {}'.format(nonfraud_count))
print('Number of Non-fraud cases are {}'.format(fraud_count))
print('Percentage of fraud cases is {}'.format(fraud_percentage))
print('--------------------------------------------')

CASE COUNT
--------------------------------------------
Total number of cases are 136243
Number of Non-fraud cases are 134827
Number of Non-fraud cases are 1416
Percentage of fraud cases is 1.05
--------------------------------------------


In [ ]:
#Replace categorical column with int value
df['STO'].value_counts()
df['PRA_PASCA'].value_counts()
df['SPEED_INET'].value_counts()

10M     47812
20M     32734
40M     22930
50M     17723
100M    10965
200M     4079
Name: SPEED_INET, dtype: int64

In [ ]:
# Replace categorical data
df['STO'] = df['STO'].replace(['PNK'], 1).replace(['BAL'], 2).replace(['MAT'], 3).replace(['SUG'], 4).replace(['TMA'], 5).replace(['SUD'], 6).replace(['WTP'], 7).replace(['ANT'], 8).replace(['MAR'], 9).replace(['PKN'], 10).replace(['TKA'], 11).replace(['SIN'], 12).replace(['KIM'], 13).replace(['MAL'], 14).replace(['BLK'], 15).replace(['BTN'], 16).replace(['SLY'], 17).replace(['JNP'], 18)
df['SPEED_INET'] = df['SPEED_INET'].replace(['200M'], 200).replace(['100M'], 100).replace(['50M'], 50).replace(['40M'], 40).replace(['20M'], 20).replace(['10M'], 10)
df['PRA_PASCA'] = df['PRA_PASCA'].replace(['PRABAYAR'], 0).replace(['PASCABAYAR'], 1)
df

,STO,SPEED_INET,USAGE,BILLING,UMUR_LANGGANAN,PRA_PASCA,Class
0,3,20,689.73,483300,15,1,0
1,1,10,404.09,217100,36,1,0
2,12,10,373.29,268700,4,1,0
3,4,20,255.92,391000,1,1,0
4,5,50,851.28,771800,51,1,0
...,...,...,...,...,...,...,...
136238,2,200,2406.40,1949600,15,1,0
136239,13,100,2535.69,944100,19,1,0
136240,1,10,410.95,279400,38,1,0
136241,1,40,697.03,552400,60,1,0


In [ ]:
df['USAGE'] = df['USAGE'].astype(int)
print (df.dtypes)

STO               int64
SPEED_INET        int64
USAGE             int64
BILLING           int64
UMUR_LANGGANAN    int64
PRA_PASCA         int64
Class             int64
dtype: object


In [ ]:
#Data Split
x = df.drop('Class', axis = 1).values
y = df['Class'].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

#Oversampling Data
ada = ADASYN(sampling_strategy=1.0, random_state=27)
x_train, y_train = ada.fit_resample(x_train, y_train)

#Min-Max Data Scalling
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
import time

start_time = time.process_time()
#Decision Tree
tree_model = DecisionTreeClassifier(max_depth = 20, criterion = 'entropy')
tree_model.fit(x_train, y_train)
tree_yhat = tree_model.predict(x_test)

end_time = time.process_time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.6f} detik")
tree_model.get_params()

Waktu eksekusi: 0.668730 detik


{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 20,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [ ]:
start_time = time.process_time()
#Random Forest
rf = RandomForestClassifier(n_estimators = 15, max_features = 1)
rf.fit(x_train, y_train)

end_time = time.process_time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.6f} detik")

start_time = time.process_time()
rf_yhat = rf.predict(x_test)
end_time = time.process_time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.6f} detik")
rf.get_params()

Waktu eksekusi: 3.619414 detik
Waktu eksekusi: 0.075624 detik


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 1,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 15,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
start_time = time.process_time()

#XGBoost
reg = xgb.XGBClassifier(max_depth=3, n_estimators=250)
reg.fit(x_train, y_train)
end_time = time.process_time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.6f} detik")

start_time = time.process_time()
y_xgb_pred = reg.predict(x_test)
print(y_test)
print(y_xgb_pred)
end_time = time.process_time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.6f} detik")

reg.get_params()

Waktu eksekusi: 6.291987 detik
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 1 0]
Waktu eksekusi: 0.115549 detik


{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 3,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 250,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [ ]:
start_time = time.process_time()
#Logistic Regression
lr = LogisticRegression()
lr.fit(x_train, y_train)

end_time = time.process_time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.6f} detik")

start_time = time.process_time()
lr_yhat = lr.predict(x_test)
end_time = time.process_time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.6f} detik")
lr.get_params()

Waktu eksekusi: 0.914128 detik
Waktu eksekusi: 0.005926 detik


{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [ ]:
start_time = time.process_time()
from sklearn.neural_network import MLPClassifier

#Data Scaling for BPNN
scaler = StandardScaler()
x_train_nn = scaler.fit_transform(x_train) #  transform
x_test_nn = scaler.transform(x_test) #  transform

#Multi Layer Perceptron Classifier
mlp = MLPClassifier(hidden_layer_sizes=125,
                    max_iter=480,
                    alpha=1e-4,
                    solver='sgd',
                    tol=1e-4,
                    random_state=1,
                    learning_rate='adaptive',
                    learning_rate_init=.1)


mlp.fit(x_train_nn, y_train)
end_time = time.process_time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.6f} detik")

start_time = time.process_time()

mlp_yhat = mlp.predict(x_test_nn)
end_time = time.process_time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.6f} detik")
mlp.get_params()

Waktu eksekusi: 713.180113 detik
Waktu eksekusi: 0.058546 detik


{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': 125,
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.1,
 'max_fun': 15000,
 'max_iter': 480,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 1,
 'shuffle': True,
 'solver': 'sgd',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, mlp_yhat)

In [ ]:
#Cross Validation

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier


# Menentukan strategi cross-validation (dalam hal ini kita menggunakan Stratified k-Fold dengan k=5)
    cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Melakukan cross-validation dan mencetak skor akurasi rata-rata
scores = cross_val_score(mlp, x_train_nn, y_train, cv=cv_strategy, scoring='accuracy')
print(f"Rata-rata skor akurasi: {np.mean(scores):.2f}")

Rata-rata skor akurasi: 0.98


In [ ]:
import itertools # advanced tools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

#Confusion Matrix
#Defining the plot function
def plot_confusion_matrix(cm, classes, title, normalize = False, cmap = plt.cm.Blues):
    title = 'Confusion Matrix of {}'.format(title)
    if normalize:
        cm = cm.astype(float) / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment = 'center',
                 color = 'white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

#Compute confusion matrix for the models
tree_matrix = confusion_matrix(y_test, tree_yhat, labels = [0, 1]) #Decision Tree
rf_matrix = confusion_matrix(y_test, rf_yhat, labels = [0, 1]) #Random Forest Tree
lr_matrix = confusion_matrix(y_test, lr_yhat, labels = [0, 1]) #Logistic Regression
mlp_matrix = confusion_matrix(y_test, mlp_yhat, labels = [0, 1]) #BPNN
xgb_matrix = confusion_matrix(y_test, y_xgb_pred, labels = [0, 1]) #XGBoost

#Plot the confusion matrix
plt.rcParams['figure.figsize'] = (6, 6)

#Decision tree
tree_cm_plot = plot_confusion_matrix(tree_matrix,
                                classes = ['Negative(0)','Positive(1)'],
                                normalize = False, title = 'Decision Tree')
plt.savefig('tree_cm_plot.png')
plt.show()

#Random forest tree
rf_cm_plot = plot_confusion_matrix(rf_matrix,
                                classes = ['Negative(0)','Positive(1)'],
                                normalize = False, title = 'Random Forest Tree')
plt.savefig('rf_cm_plot.png')
plt.show()

# Logistic regression
lr_cm_plot = plot_confusion_matrix(lr_matrix,
                                classes = ['Negative(0)','Positive(1)'],
                                normalize = False, title = 'Logistic Regression')
plt.savefig('lr_cm_plot.png')
plt.show()

# BPNN
mlp_cm_plot = plot_confusion_matrix(mlp_matrix,
                                classes = ['Negative(0)','Positive(1)'],
                                normalize = False, title = 'Backpropagation Neural Network')
plt.savefig('lr_cm_plot.png')
plt.show()

#XGBoost
xgb_cm_plot = plot_confusion_matrix(xgb_matrix,
                                classes = ['Negative(0)','Positive(1)'],
                                normalize = False, title = 'XG Boost')
plt.savefig('xgb_cm_plot.png')
plt.show()

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

#Scoring
print('Precision Decision Tree {}'.format(precision_score(y_test, tree_yhat)))
print('Recall Decision Tree {}'.format(recall_score(y_test, tree_yhat)))
print('F1 Score Decision Tree {}'.format(f1_score(y_test, tree_yhat)))
print('Accuracy Score Decision Tree {}'.format(accuracy_score(y_test, tree_yhat)))
print('_______________________________________________________________________________')
print('Precision Random Forest {}'.format(precision_score(y_test, rf_yhat)))
print('Recall Random Forest {}'.format(recall_score(y_test, rf_yhat)))
print('F1 Score Random Forest {}'.format(f1_score(y_test, rf_yhat)))
print('Accuracy Score Random Forest {}'.format(accuracy_score(y_test, rf_yhat)))
print('_______________________________________________________________________________')
print('Precision Logistic Regression {}'.format(precision_score(y_test, lr_yhat)))
print('Recall Logistic Regression {}'.format(recall_score(y_test, lr_yhat)))
print('F1 Score Logistic Regression {}'.format(f1_score(y_test, lr_yhat)))
print('Accuracy Score Logistic Regression {}'.format(accuracy_score(y_test, lr_yhat)))
print('_______________________________________________________________________________')
print('Precision Backpropagation Neural Network {}'.format(precision_score(y_test, mlp_yhat)))
print('Recall Backpropagation Neural Network {}'.format(recall_score(y_test, mlp_yhat)))
print('F1 Score Backpropagation Neural Network {}'.format(f1_score(y_test, mlp_yhat)))
print('Accuracy Score Backpropagation Neural Network {}'.format(accuracy_score(y_test, mlp_yhat)))
print('_______________________________________________________________________________')
print('Precision XGBoost{}'.format(precision_score(y_test, y_xgb_pred)))
print('Recall XGBoost {}'.format(recall_score(y_test, y_xgb_pred)))
print('F1 Score XGBoost {}'.format(f1_score(y_test, y_xgb_pred)))
print('Accuracy Score XGBoost {}'.format(accuracy_score(y_test, y_xgb_pred)))

In [ ]:
#Train Score
print('Precision Decision Tree {}'.format(precision_score(y_train, tree_model.predict(x_train))))
print('Recall Decision Tree {}'.format(recall_score(y_train, tree_model.predict(x_train))))
print('F1 Score Decision Tree {}'.format(f1_score(y_train, tree_model.predict(x_train))))
print('Accuracy Score Decision Tree {}'.format(accuracy_score(y_train, tree_model.predict(x_train))))
print('_______________________________________________________________________________')
print('Precision Random Forest {}'.format(precision_score(y_train, rf.predict(x_train))))
print('Recall Random Forest {}'.format(recall_score(y_train, rf.predict(x_train))))
print('F1 Score Random Forest {}'.format(f1_score(y_train, rf.predict(x_train))))
print('Accuracy Score Random Forest {}'.format(accuracy_score(y_train, rf.predict(x_train))))
print('_______________________________________________________________________________')
print('Precision Logistic Regression {}'.format(precision_score(y_train, lr.predict(x_train))))
print('Recall Logistic Regression {}'.format(recall_score(y_train, lr.predict(x_train))))
print('F1 Score Logistic Regression {}'.format(f1_score(y_train, lr.predict(x_train))))
print('Accuracy Score Random Forest {}'.format(accuracy_score(y_train, lr.predict(x_train))))
print('_______________________________________________________________________________')
print('Precision Backpropagation Neural Network {}'.format(precision_score(y_train, mlp.predict(x_train_nn))))
print('Recall Backpropagation Neural Network {}'.format(recall_score(y_train, mlp.predict(x_train_nn))))
print('F1 Score Backpropagation Neural Network {}'.format(f1_score(y_train, mlp.predict(x_train_nn))))
print('Accuracy Score Random Forest {}'.format(accuracy_score(y_train, mlp.predict(x_train_nn))))
print('_______________________________________________________________________________')
print('Precision XGBoost{}'.format(precision_score(y_train, reg.predict(x_train))))
print('Recall XGBoost {}'.format(recall_score(y_train, reg.predict(x_train))))
print('F1 Score XGBoost {}'.format(f1_score(y_train, reg.predict(x_train))))
print('Accuracy Score XGBoost {}'.format(accuracy_score(y_train, reg.predict(x_train))))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

#Defining ROC AUC
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

#Defining probs of each model
#Decision Tree
tree_model_prob = tree_model.predict_proba(x_test)
tree_model_prob = tree_model_prob[:, 1]
auc_tree_model = roc_auc_score(y_test, tree_model_prob)
print('AUC: %.2f' % auc_tree_model)
fpr_tree_model, tpr_tree_model, thresholds_tree_model = roc_curve(y_test, tree_model_prob)
#Random Forest
rf_prob = rf.predict_proba(x_test)
rf_prob = rf_prob[:, 1]
auc_rf = roc_auc_score(y_test, rf_prob)
print('AUC: %.2f' % auc_rf)
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, rf_prob)
#Logistic Regression
lr_prob = lr.predict_proba(x_test)
lr_prob = lr_prob[:, 1]
auc_lr = roc_auc_score(y_test, lr_prob)
print('AUC: %.2f' % auc_lr)
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, lr_prob)
#BPNN
mlp_prob = mlp.predict_proba(x_test_nn)
mlp_prob = mlp_prob[:, 1]
auc_mlp = roc_auc_score(y_test, mlp_prob)
print('AUC: %.2f' % auc_mlp)
fpr_mlp, tpr_mlp, thresholds_mlp = roc_curve(y_test, mlp_prob)


#Plotting roc
#Decision Tree
plot_roc_curve(fpr_tree_model, tpr_tree_model)
#Random Forest
plot_roc_curve(fpr_rf, tpr_rf)
#Logistic Regression
plot_roc_curve(fpr_lr, tpr_lr)
#BPNN
plot_roc_curve(fpr_mlp, tpr_mlp)
